<a href="https://colab.research.google.com/github/uwsampl/tutorial/blob/master/notebook/06_TVM_Tutorial_MicroTVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Please run the following block to ensure TVM is setup for *this notebook*.  Each notebook may have its own runtime.

In [0]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    ! gsutil cp "gs://tvm-fcrc-binariesd5fce43e-8373-11e9-bfb6-0242ac1c0002/tvm.tar.gz" /tmp/tvm.tar.gz
    ! mkdir -p /tvm
    ! tar -xf /tmp/tvm.tar.gz --strip-components=4 --directory /tvm
    ! ls -la /tvm
    ! bash /tvm/package.sh
    # Add TVM to the Python path.
    import sys
    sys.path.append('/tvm/python')
    sys.path.append('/tvm/topi/python')
    ! pip install mxnet
else:
    print('Notebook executing locally, skipping Colab setup ...')

# μTVM: Deep Learning on Bare-Metal Devices

In [0]:
import os
import pprint

import numpy as np
import tvm
from tvm.contrib import graph_runtime, util
from tvm import relay
import tvm.micro as micro

The proliferation of low-cost, AI-powered consumer devices has lead to widespread interest in "bare-metal" (low-power, often without an operating system) devices among ML researchers and practitioners.

μTVM brings deep learning code generated by TVM to bare-metal devices that lack mature software stacks or do not have an operating system. Here's a high-level overview of how μTVM operates:

This tutorial aims to provide a broad understanding of how μTVM works, and primarily focuses on how you can use μTVM to run TVM-generated code on your own bare-metal device.

In this notebook, we're going to cover:

*   the C code generation backend
*   the cross-compiler interface
*   the low-level device interface
*   the graph runtime
*   a demo of ResNeT-18 using the graph runtime


There are three requirements for using μTVM:
- A TVM model written in Python
- A low-level device interface to the target device
- A cross-compiler for the target device

**Note**: We have only tested μTVM using a Linux host machine.  It may or may not work on Mac.

**Disclaimer**: μTVM is still experimental and its API may change in the future.

## C Codegen

If we're going to work with bare-metal devices, we need some type of code generation backend that targets them.  At first glance, LLVM seems like a great option, but unfortunately, we can't use it.  Bare-metal devices often lack support for common languages and IRs.

However, most bare-metal devices ***do*** support C.  We have taken advantage of this fact by developing a C code generation backend for TVM.  With this backend, any programs expressible in TVM's low-level IR can be compiled into C.


In [0]:
def build_c_module(func):
  """Builds a C source module by compiling `func` with Relay."""
  # Note: μTVM currently does not support vectorized instructions.
  with tvm.build_config(disable_vectorize=True):
    # We set `target` to "c", and the build is now routed to the C backend.
    _, src_mod, _ = relay.build(func, target='c', params={})
  return src_mod

Now, let's write a simple Relay program and examine the generated code.



In [0]:
# First, we use Relay to construct our function which computes `x + y`.
ty = relay.TensorType(shape=(1024,), dtype='float32')
x = relay.var('x', ty)
y = relay.var('y', ty)
func = relay.Function([x, y], relay.add(x, y))
print(func)

In [0]:
# Then build it into a C source module.
src_mod = build_c_module(func)
# Now, we can see the source that was generated.
print(src_mod.get_source())

## The Cross-Compiler Interface
With the ability to generate C, we need access to a cross-compiler toolchain for our device, in order to create binaries for it.

How you obtain the toolchain will vary, depending on the device you're targetting.  Once you have a toolchain, you need to make sure it's accessible from a terminal by adding it to your system's PATH variable.

All of the binaries in the toolchain should have a common prefix.  For example, if you're targetting RISC-V, you might want "gcc" to be named "riscv-gcc", "ld" to be named "riscv-ld", etc., and in this case, the prefix is "riscv-".  Having a common prefix is important, because we use this prefix to tell μTVM which binaries to use for binary compilation/manipulation.

In the example below, we create a session using the host emulated device type, which creates a region of memory on the host machine to emulate device memory.  Because this device type is running on the host machine, we don't need a cross-compiler, and in fact, we can use plain old "gcc".  In this case, there is no prefix on the toolchain, so we use the empty string.

In [0]:
device_type = 'host'
toolchain_prefix = ''
with micro.Session(device_type, toolchain_prefix) as sess:
  # Do μTVM things...
  pass

## The Low-Level Device Interface
Now we have the ability to generate binaries for our device, but we don't yet have a method of loading and running these binaries onto the device.  And even if we could run it, we couldn't read the outputs.  

To satisfy these requirements, we require an implementation of a low-level device interface for every device in μTVM.  The low-level device interface exposes three important operations to interact with the MicroDevice: 
- Reading from device memory
- Writing to device memory
- Beginning function execution

This interface appears in C++ as

<img src="https://i.imgur.com/beF6Rm3.png" alt="low-level device interface" width="500"/>

In the above interface, `DevBaseOffset` is a wrapper type for a `uintptr_t` that represents an offset from the base address of the device.  The full definition of the inteface can be found [here](https://github.com/uwsampl/tvm/blob/tutorial/src/runtime/micro/low_level_device.h).

To add support for your own MicroDevice in μTVM, you simply need to implement the low-level device interface for your MicroDevice.  Doing so will require some form of communication with the device.  For instance, communication may occur over [JTAG](https://en.wikipedia.org/wiki/JTAG) or over a network connection.

### Host Low-Level Device
To give an example of an implementation of this interface, we will walk through the already-existing `HostLowLevelDevice` (found [here](https://github.com/uwsampl/tvm/blob/tutorial/src/runtime/micro/host_low_level_device.cc)), because it's implementation is relatively simple.

This device uses an allocated region of memory on the host machine to simulate a MicroDevice. This device provides μTVM users with an easy setup to experiment with, without having to communicate with and debug external MicroDevices.

#### Constructor
To create a host device, we call `mmap` to allocate a memory region that is at least as large as the requested device size, and we assign read/write/execute permissions to that region.  The base address of the device is just the start address of the region.

<img src="https://i.imgur.com/FVNSVnV.png" alt="constructor implementation" width="700"/>

#### Destructor
To destroy a host device, we simply unmap the `mmap`'d region.

<img src="https://i.imgur.com/rthH1Cv.png" alt="destructor implementation" width="400"/>

#### Read
To read from the host device, we convert the device offset into a host virtual address, then use `memcpy` to transfer from the mapped memory into the output buffer `buf`.

<img src="https://i.imgur.com/77lRlzZ.png" alt="read implementation" width="500"/>

#### Write
Writing to the host device is similar to reading, except now `buf` serves as an input buffer.

<img src="https://i.imgur.com/SEm4UWQ.png" alt="write implementation" width="500"/>

#### Execute
For function execution, we are given an offset to the target function, but also a breakpoint indicating the memory offset at which code execution should stop.

We don't need to use `breakpoint` here, because with the host device, control flow never leaves the host machine, so we can simply call the function as we would any other function.

For real devices, we may need to use `breakpoint` to stop execution, because otherwise, the device may continue executing indefinitely.

<img src="https://i.imgur.com/psuYqrt.png" alt="execute implementation" width="500"/>

## Graph Runtime Execution

The primary execution strategy for running models with μTVM is the graph runtime.

When we use the graph runtime, the host drives the control flow of the model.  For each node in the graph, the host calls out to the device to execute the corresponding operator for that node.

Note that this means models do not run entirely on the device (i.e., they aren't self-hosted).  Fully self-hosted models are not currently supported, as this would require using the Relay ahead-of-time compiler, which is still experimental.


In [0]:
shape = (1024,)
dtype = 'float32'
    
# Construct Relay program.
x = relay.var('x', relay.TensorType(shape=shape, dtype=dtype))
xx = relay.multiply(x, x)
z = relay.add(xx, relay.const(1.0))
func = relay.Function([x], z)
print(func)

In [0]:
with micro.Session('host', '') as sess:
  mod, params = sess.build(func)
  mod.set_input(**params)

  x_in = np.random.uniform(size=shape[0]).astype(dtype)
  print(f'input:\t\t\t{x_in}')
  # Run with `x_in` as the input. 
  mod.run(x=x_in)
  result = mod.get_output(0).asnumpy()

  expected = (x_in * x_in) + 1.0
  print()
  print(f'expected result:\t{expected}')
  print(f'μTVM result:\t\t{result}')
  print()
  if np.array_equal(expected, result):
    print('Looks good!')
  else:
    print('Something\'s wrong!')

## ResNet-18 Demo



In [0]:
import io
import mxnet as mx
from mxnet.gluon.model_zoo.vision import get_model
from mxnet.gluon.utils import download
from PIL import Image
import IPython.display

RESNET_INPUT_IMG_SHAPE = (1, 3, 224, 224)

def get_image():
  img_name = 'cat.png'
  download('https://github.com/dmlc/mxnet.js/blob/master/data/cat.png?raw=true',
           img_name)
  image = Image.open(img_name)
  return image


def show_image(image):
  img_byte_arr = io.BytesIO()
  image.save(img_byte_arr, format='png')
  img_byte_arr = img_byte_arr.getvalue()
  IPython.display.display(IPython.display.Image(img_byte_arr))


def convert_image(image):
  dtype = 'float32'
  image = image.resize(RESNET_INPUT_IMG_SHAPE[2:])
  image = np.array(image) - np.array([123., 117., 104.])
  image /= np.array([58.395, 57.12, 57.375])
  image = image.transpose((2, 0, 1))
  image = image[np.newaxis, :]
  image = tvm.nd.array(image.astype(dtype))
  return image


def get_resnet():    
  block = get_model('resnet18_v1', pretrained=True)
  module, params = relay.frontend.from_mxnet(
    block, shape={'data': RESNET_INPUT_IMG_SHAPE})
  func = module['main']
  return func, params
  
  
# Fetch a mapping from class IDs to human-readable labels.
synset_url = ''.join(['https://gist.githubusercontent.com/zhreshold/',
                      '4d0b62f3d01426887599d4f7ede23ee5/raw/',
                      '596b27d23537e5a1b5751d2b0481ef172f58b539/',
                      'imagenet1000_clsid_to_human.txt'])
synset_name = 'synset.txt'
download(synset_url, synset_name)
with open(synset_name) as f:
  synset = eval(f.read())

    
def get_predictions(mod):
  output = mod.get_output(0).asnumpy()[0]
  top_k = output.argsort()[-3:][::-1]
  predictions = list(map(lambda x: synset[x], top_k))
  return predictions

It's great that we can prototype models and immediately run them on a device, but the function we just ran was pretty simple.  Let's try a more interesting example.

In this section, we will run ResNet-18 on μTVM to perform image recognition.  And our example input is...

In [0]:
image = get_image()
show_image(image)
# Convert image to a format the model can understand.
image = convert_image(image)

...a cat!

Luckily, MxNet is one of the many frameworks that TVM can import models from, so we can use a pretrained ResNet-18 model from the Gluon Model Zoo.

Below, we grab a relay function that expresses our model, along with its set of pretrained parameters.


In [0]:
resnet, params = get_resnet()
print(resnet)
print()
pprint.pprint(list(params.keys()))

With that, we can create a μTVM session, build a graph runtime module, then run the model.

In [0]:
with micro.Session('host', '') as sess:
  print('Building...')
  module, params = sess.build(resnet, params=params)
  module.set_input(**params)
  print()
  print('Running...', end='')
  # Execute with `image` as the input.
  module.run(data=image)
  print()
  print('Predictions')
  for i, prediction in enumerate(get_predictions(module)):
    print(f'  {i + 1}: {prediction}')

# Additional Resources

For more details on how μTVM is implemented, you can look into the following pull requests on the [TVM GitHub](https://github.com/dmlc/tvm) repository: 

- [C code generation backend](https://github.com/dmlc/tvm/pull/2161)
- [μTVM RFC](https://github.com/dmlc/tvm/issues/2563) (somewhat outdated)
- [μTVM on host-emulated device](https://github.com/dmlc/tvm/pull/3227)
- μTVM on RISC-V: Will be upstreamed in a few weeks!



